In [0]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim



#loading the data and preprocessing it
mnist_train= torchvision.datasets.MNIST("./data",train=True, download=True, transform=transforms.Compose([transforms.ToTensor()])) # transform the data to a tensor
mnist_test= torchvision.datasets.MNIST("./data",train=False, download=True, transform=transforms.Compose([transforms.ToTensor()])) # transform the data to a tensor

#how are we gonna iterate over the data?
train_loader= torch.utils.data.DataLoader(mnist_train,batch_size=4,shuffle=True) #batch_size : process the data in batches and make a better generalization
test_loader= torch.utils.data.DataLoader(mnist_test,batch_size=4,shuffle=False)  #shuffling the data makes a better generalization

#every training data has 4 images and 4 labels

## NN Definition
class Net(nn.Module):

  def __init__(self):
    super().__init__() #initialize your network
    self.fc1=nn.Linear(784, 64) #28*28 -> first flattened image and fully connected layer
    self.fc2=nn.Linear(64, 64) # second layer
    self.fc3=nn.Linear(64, 64) #third layer
    self.fc4=nn.Linear(64, 10) #output layer has 10 neurons
    
  def forward(self,x):
    x=F.relu(self.fc1(x)) # apply the activation function
    x=F.relu(self.fc2(x)) # apply the activation function
    x=F.relu(self.fc3(x)) # apply the activation function
    x=self.fc4(x) # apply the activation function either zero or one ! so one of the node is fired 
    return F.log_softmax(x,dim=1)
    #this function will return for each picture: 10 nodes (prediction value of each label)
   

In [12]:
# Driver code
net=Net()
optimizer=optim.Adam(net.parameters(),lr=0.001) #get adjustable parameters
total_epoch=30 #How many times we pass our full data (the same data)

#training
correct=0
total=0
for cur_epoch in range(total_epoch):
  for data in train_loader:
    #every data consits of 4 (batch_size) umages
    X,y=data #picture(X batch_size), label(X batch_size) -> #batch size comes first
    net.zero_grad()  
    output = net(X.view(-1,784))  
    loss = F.nll_loss(output, y)
    loss.backward()  
    optimizer.step()
    #calculate how many write do you have in every training data until the end of all training datas
    #output is Batch_size*10 tensor
    for k, i in enumerate(output): # the output is batch_size* 10 tensor   # k is the index of the data # i the data itself
        if torch.argmax(i) == y[k]: # in every row find the highest prediction index and compare it to y[k]
                correct += 1
        total += 1
  #done with one epoch
  acc=correct/total
  if((cur_epoch+1)%(total_epoch*0.1)==0):
     print('Epoch {}/{}: Accuracy {}'.format(cur_epoch+1, total_epoch, acc))
  correct=0
  total=0



Epoch 3/30: Accuracy 0.9704166666666667
Epoch 6/30: Accuracy 0.97865
Epoch 9/30: Accuracy 0.9820333333333333
Epoch 12/30: Accuracy 0.9854
Epoch 15/30: Accuracy 0.9868333333333333
Epoch 18/30: Accuracy 0.9882833333333333
Epoch 21/30: Accuracy 0.9885666666666667
Epoch 24/30: Accuracy 0.9886833333333334
Epoch 27/30: Accuracy 0.9897
Epoch 30/30: Accuracy 0.9903333333333333


In [13]:
#validating
correct =0
total=0
with torch.no_grad(): # no gradient
  for data in test_loader:
      X, y = data # store the images in X and labels in y
      output = net(X.view(-1,784)) #send the 4 images
      #print(output)
      for k, i in enumerate(output): # the output is 4* 10 ARRAY
          if torch.argmax(i) == y[k]: # in every row find the highest prediction and comprae its index
              correct += 1
          total += 1

print("Test Accuracy: ", correct/total)

Test Accuracy:  0.9723
